To aid in generating an equity research report for Bendigo and Adelaide bank, first a fundimental analysis is performed by defining relevent functions to calculate key profitability, liquidity and efficiency and valuation ratios. 

Runs instance of DDM.py to create model

In [ ]:
def net_interest_margin(net_interest, earning_assets):
    nim = net_interest / earning_assets
    return nim
def calculate_dividend_yield(dividend, last_price):
    if last_price == 0:
        return 0
    return dividend / last_price
def calculate_pe_ratio(last_price, eps):
    if eps == 0:
        return 0
    return last_price / eps
def calculate_eps(earnings, shares_outstanding):
    if shares_outstanding == 0:
        return 0
    return earnings / shares_outstanding
def calculate_roa(earnings, total_assets):
    if total_assets == 0:
        return 0
    return earnings / total_assets
def calculate_pe_ratio(last_price, eps):
    if eps == 0:
        return 0
    return last_price / eps
def calculate_eps(earnings, shares_outstanding):
    if shares_outstanding == 0:
        return 0
    return earnings / shares_outstanding

To provide 

Specific 

Comentary for the company was developed from all previous analysis to include an executive summary, business overview, investment theses and analysis of key finanical ratios. 

In [253]:
from fpdf import FPDF
from html.parser import HTMLParser
import pandas as pd
import openpyxl

class PDF(FPDF):
    def header(self):
        banner_width = self.w - self.l_margin - self.r_margin
        self.image('test.png', x=self.l_margin, y=0, w=banner_width, h=20)
        self.set_y(25)

    def footer(self):
        self.set_y(-12)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 8, f'Page {self.page_no()}', align='C')


class ColumnHTMLParser(HTMLParser):
    def __init__(self, pdf, x, y, col_width, line_height=6):
        super().__init__()
        self.pdf = pdf
        self.x = x
        self.y = y
        self.col_width = col_width
        self.line_height = line_height
        self.current_tag = None
        self.pdf.set_xy(x, y)

    def handle_starttag(self, tag, attrs):
        self.current_tag = tag
        if tag == 'h1':
            self.pdf.set_font('Arial', 'B', 16)
            self.pdf.set_xy(self.x, self.pdf.get_y())
        elif tag == 'h2':
            self.pdf.set_font('Arial', 'B', 13)
            self.pdf.set_xy(self.x, self.pdf.get_y())
        elif tag == 'p':
            self.pdf.set_font('Arial', '', 11)
            self.pdf.set_xy(self.x, self.pdf.get_y())

    def handle_endtag(self, tag):
        if tag == 'p':
            self.pdf.ln(2)
        self.current_tag = None

    def handle_data(self, data):
        self.pdf.multi_cell(self.col_width, self.line_height, data)
        self.pdf.set_x(self.x)  # stay in column after line wraps



# Set up PDF
pdf = PDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Column dimensions
total_width = pdf.w - pdf.l_margin - pdf.r_margin
col1_w = total_width * 2 / 3
col2_w = total_width * 1 / 3
line_height = 6

# Starting position after header
y_start = pdf.get_y()


Right hand column content

In [254]:
# Right column content
# --------- RIGHT COLUMN (Insert Image) ---------
x_right = pdf.l_margin + col1_w + 5  # Start of right column
y_right_start = y_start
col2_actual_width = col2_w - 5  # Narrow a little for margin space

# Set light blue color for the background
pdf.set_fill_color(230, 240, 255)  # Very light blue

# Draw the rectangle
pdf.rect(x_right, y_right_start, col2_actual_width, 350, style='F')
pdf.set_xy(pdf.l_margin + col1_w + 5, y_start)
right_x = pdf.get_x()
right_y = pdf.get_y()

# Load the workbook
wb = load_workbook('BENDIGO.xlsx', data_only=True)
ws = wb.active  # or wb['YourSheetName']

# Now manually read D9:J19 preserving formatting
data = []
for row in ws.iter_rows(min_row=11, max_row=16, min_col=4, max_col=6):
    row_data = []
    for cell in row:
        value = cell.value
        number_format = cell.number_format
        # Apply number formatting manually
        if value is None:
            display_value = ''
        elif '0%' in number_format or number_format.endswith('%'):
            # Format as percentage
            display_value = f"{value * 100:.1f}%"
        elif '$' in number_format or 'Accounting' in number_format:
            display_value = f"${value:,.2f}"
        elif '#,##0' in number_format or ',' in number_format:
            display_value = f"{value:,.0f}"
        else:
            display_value = value

        row_data.append(display_value)
    data.append(row_data)

print(data)
df = pd.DataFrame(data)
print(df)

pdf.set_xy(pdf.l_margin + col1_w + 5, y_start)
right_x = pdf.get_x()
right_y = pdf.get_y()

pdf.set_font('Arial', '', 4)

col_width = (col2_w - 5) / df.shape[1]  # number of columns
row_height = 5
pdf.set_y(right_y + 10)

df.columns = ["Metric", "Unit", "Value"]
pdf.set_x(right_x)
# --------- Draw HEADER row ----------
pdf.set_font('Arial', 'B', 4)  # Bold font
for col_name in df.columns:
    pdf.cell(col_width, row_height, str(col_name), border='B', align='C')
pdf.ln(row_height)
pdf.set_x(right_x)

# --------- Draw BODY rows ----------
pdf.set_font('Arial', '', 4)  # Back to normal font
for i in range(len(df)):
    for j in range(len(df.columns)):
        value = str(df.iloc[i, j])
        pdf.cell(col_width, row_height, value, border='B', align='C')
    pdf.ln(row_height)
    pdf.set_x(right_x)

right_x = pdf.get_x()
right_y = pdf.get_y()

# Load image into right column
image_width = col2_w - 10  # leave small gap

# Move cursor after the image
pdf.set_y(right_y + 50)  # Adjust 50 to be the height of your image (approx.)
pdf.image('cropped_financial_results.png', x=right_x+2.5, y=right_y+2.5, w=image_width)

pdf.set_y(right_y + 35)  # Adjust 50 to be the height of your image (approx.)
pdf.set_x(right_x+10)
right_x = pdf.get_x()
right_y = pdf.get_y()


from Ratios_and_Metrics import financial_metrics_bendigo

df = financial_metrics_bendigo()
# --------- Draw HEADER row ----------
pdf.set_font('Arial', 'B', 4)  # Bold font
for col_name in df.columns:
    pdf.cell(col_width, row_height, str(col_name), border='B', align='C')
pdf.ln(row_height)
pdf.set_x(right_x)


# --------- Draw BODY rows ----------
pdf.set_font('Arial', '', 4)  # Back to normal font
for i in range(len(df)):
    for j in range(len(df.columns)):
        value = str(df.iloc[i, j])
        pdf.cell(col_width, row_height, value, border='B', align='C')
    pdf.ln(row_height)
    pdf.set_x(right_x)

right_x = pdf.get_x()
right_y = pdf.get_y()




[['Valuation Date', '[Date]', datetime.datetime(2025, 4, 25, 0, 0)], ['Last Share Price', '[$]', 10.88], ['Intrinsic Value', '[$]', 13.89], ['Diluted Shares Outstanding', '[m]', '567'], ['Cost of Equity', '[%]', '7.9%'], ['Terminal Growth Rate', '[%]', '2.1%']]
                            0       1                    2
0              Valuation Date  [Date]  2025-04-25 00:00:00
1            Last Share Price     [$]                10.88
2             Intrinsic Value     [$]                13.89
3  Diluted Shares Outstanding     [m]                  567
4              Cost of Equity     [%]                 7.9%
5        Terminal Growth Rate     [%]                 2.1%
{'Dividend Yield (%)': np.float64(2.69), 'Earnings Per Share (EPS)': 0.96, 'Price to Earnings (P/E)': np.float64(11.58), 'Return on Assets (ROA %)': np.float64(0.55), 'Return on Equity (ROE %)': 7.68, 'Profit Margin (%)': 27.03, 'Operating Margin (%)': 33.38, 'Price to Book (P/B)': np.float64(0.89), 'Dividend Payout Ratio (

In [255]:
# Load the commentary from file
with open("commentary.txt", "r", encoding="utf-8") as f:
    html_content = f.read()

# --------- LEFT COLUMN (Formatted HTML) ---------
pdf.set_xy(pdf.l_margin, y_start)
html_parser = ColumnHTMLParser(pdf, x=pdf.l_margin, y=y_start, col_width=col1_w, line_height=line_height)
html_parser.feed(html_content)
y_left_end = pdf.get_y()

# Ensure bottom spacing is correct
pdf.set_y(max(y_left_end, y_right_end) + 5)

# Save it
pdf.output('BEN.pdf')

''

To provide context for the recent financial performance of the company in 1H25, an image was scrapped from the companies recent announcement of key regulatory ratios and indicators of loan book health. These statistics could not be inferred from the yFinance data and thus provided unique value to the report.

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Step 1: Define the URL
url = 'https://www.bendigobank.com.au/media/financial-results-for-the-half-year-ended-31-december-2024/'

# Step 2: Fetch the webpage
response = requests.get(url)
response.raise_for_status()

# Step 3: Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Step 4: Find the image with the specific src
target_src = '/contentassets/f4ca4760f8394c3582ce67b6a6334bbc/price-table.png'
img_tag = soup.find('img', {'src': target_src})

# Step 5: If found, build the full image URL and download
if img_tag:
    img_url = urljoin(url, img_tag['src'])  # Build absolute URL
    print(f"Image URL: {img_url}")
    
    # Optional: Download the image
    img_response = requests.get(img_url)
    img_response.raise_for_status()
    with open('data_table.png', 'wb') as f:
        f.write(img_response.content)

from PIL import Image  # <-- add this!

# Right column: image instead of placeholder text
ORIGINAL_IMAGE = 'financial_results.png'
CROPPED_IMAGE = 'cropped_financial_results.png'  # temp file

# Function to crop the left half of an image
def crop_left_half(input_path, output_path):
    with Image.open(input_path) as img:
        width, height = img.size
        # Crop (left, top, right, bottom)
        left_half = img.crop((0, 0, width // 2 - 5, height))
        left_half.save(output_path)

# Crop the image before doing anything else
crop_left_half(ORIGINAL_IMAGE, CROPPED_IMAGE)

Image URL: https://www.bendigobank.com.au/contentassets/f4ca4760f8394c3582ce67b6a6334bbc/price-table.png


In [257]:
import requests
from bs4 import BeautifulSoup

# Reading in AFR URL for Bendigo Website
AFR = 'https://www.afr.com/company/asx/ben'
RawHTML = requests.get(AFR)

# Parse the raw HTML using BeautifulSoup
parse = BeautifulSoup(RawHTML.content, 'html.parser')

# Find recent AFR headlines by filtering to publication-specific format
headlines = parse.find_all('h3', class_='-_73414cb90844eda9-headline')

# Pull and print the most recent 10 headlines
for headline in headlines[:10]:
    print(headline.get_text(strip=True))

Banks campaign for cash levy to avoid Armaguard ‘monopoly’
Slash stamp duty to encourage retirees to downsize: ING
Bank profit falls send uneasy investors into panic mode
The ASX queries Bendigo Bank’s disclosures after share price tumble
What we learnt: Investors just handed out the biggest beating of profit season
Surge in Bendigo Bank lending crunches margins, sends shares sliding
Westpac boss supports RBA rate cut, says it will deliver needed relief
Banks would face new loan regime under Coalition amid ESG pushback
Credit to Chalmers for non-partisan RBA picks
Macquarie, Westpac and CBA big losers from $350m rural bank levy
